# Starbucks Capstone Challenge

**Author:** Yousef Ismail Ahmed  
**Objective:** Prepare a clean, modeling-ready dataset to predict customer offer completion.

---

## 1. Business Objective
The objective of this project is to combine **transactional**, **demographic**, and **offer-related** data to build a dataset suitable for predictive modeling.  
The final goal is to predict whether a customer will complete a promotional offer sent through the Starbucks Rewards mobile application.

---

## 2. Project Scope
In this notebook, the following steps are performed:

- Understand the dataset and business context
- Load and inspect the data
- Merge multiple data sources into a unified dataset
- Clean and preprocess the data
- Prepare a final dataset ready for modeling

---

## 3. About the Dataset

### Overview
This dataset contains **simulated customer behavior data** from the Starbucks Rewards mobile application. Starbucks periodically sends promotional offers to users in order to increase engagement and purchase activity.

Offers vary in type and include:
- **Informational offers** (advertisements)
- **Discount offers**
- **BOGO (Buy One Get One Free) offers**

Not all customers receive the same offers, and some customers may not receive any offer during certain periods.  
This variability introduces a realistic marketing challenge:  
**identifying which customer segments respond best to which types of offers**.
.

Each offer has a defined **validity period**, during which the customer is assumed to be influenced by the offer.  
Even informational offers include a validity duration, representing the time window during which the advertisement may impact customer behavior.

The dataset includes:
- **Transactional data** showing purchases made on the app, including timestamps and spending amounts
- **Offer interaction data**, recording when offers are received, viewed, and completed
- **Demographic data**, describing customer characteristics such as age, gender, and income

Although simplified compared to the real Starbucks ecosystem (which includes multiple products), this dataset effectively models real-world customer–offer interactions and provides a strong foundation for behavioral analysis and predictive modeling.

---

## 4. Dataset Description

### `portfolio.csv`
Contains information about promotional offers.

- `id` (string): Offer ID  
- `offer_type` (string): Type of offer (BOGO, discount, informational)  
- `difficulty` (int): Minimum required spend to complete an offer  
- `reward` (int): Reward given upon offer completion  
- `duration` (int): Offer validity period (in days)  
- `channels`: Communication channels used to deliver the offer  

---

### `profile.csv`
Contains customer demographic information.

- `id` (string): Customer ID  
- `age` (int): Age of the customer  
- `gender` (string): Gender of the customer (M, F, or O)  
- `income` (float): Customer income  
- `became_member_on` (int): Date the customer joined the rewards program  

---

### `transcript.csv`
Contains event-level customer activity.

- `event` (string): Type of event (transaction, offer received, offer viewed, offer completed)  
- `person` (string): Customer ID  
- `time` (int): Time in hours since the start of the experiment  
- `value` (dictionary): Contains offer ID or transaction amount depending on the event  

---

## 5. Outcome
The final output of this notebook is a **clean, aggregated dataset** where each row represents a unique **(customer, offer)** pair.  
**This insight can support data-driven marketing decisions and personalized
offer targeting.**




# I) Import Libraries & Read Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
%matplotlib inline


pd.set_option("display.max_columns",None)

portfolio = pd.read_csv('../../03_Data/Raw_Data/portfolio.csv')
profile = pd.read_csv('../../03_Data/Raw_Data/profile.csv')
transcript = pd.read_csv('../../03_Data/Raw_Data/transcript.csv')

portfolio.head(), profile.head(), transcript.head()

(   reward channels/0 channels/1 channels/2  difficulty  duration  \
 0      10      email     mobile     social          10         7   
 1      10        web      email     mobile          10         5   
 2       0        web      email     mobile           0         4   
 3       5        web      email     mobile           5         7   
 4       5        web      email        NaN          20        10   
 
       offer_type                                id channels/3  
 0           bogo  ae264e3637204a6fb9bb56bc8210ddfd        NaN  
 1           bogo  4d5c57ea9a6940dd891ad53e9dbe8da0     social  
 2  informational  3f207df678b143eea3cee63160fa8bed        NaN  
 3           bogo  9b98b8c7a33c4b65b9aebfe6a799e6d9        NaN  
 4       discount  0b1e1539f2cc45b7b9fa7c272da2e1d7        NaN  ,
   gender  age                                id  became_member_on    income
 0    NaN  118  68be06ca386d4c31939f3a4f0e3dd783          20170212       NaN
 1      F   55  0610b486422d4921ae7d2bf

In [2]:
portfolio.shape,profile.shape,transcript.shape

((10, 9), (17000, 5), (59980, 7))

**Initial Data Inspection At this stage, the datasets are loaded and briefly inspected to understand their structure, size, and main features before proceeding with merging and cleaning.** 

-------------------------

# Merge

**After merging, the goal is to understand each dataset before combining them. To better understand customer behavior and offer distribution, we start by examining the frequency of customers and offers in the transcript dataset.**

In [3]:
transcript.person.value_counts()

person
a95c94e9e8444d3ea4e10d2b96c6bba4    15
26de843918834f53b73844559b4b90f2    15
e67ada5fe1e240879e7a104257b196f8    15
f3cd5ab3f77e491a91d854e3f04d04c3    15
e4087b3f387b47e38d58e920f779056b    15
                                    ..
c4556c7bb4ab4c9d8e5fbf15c450b2b8     1
de2fdfba151e4fa6a940412e771316c4     1
537d1682c42740218162e3bca53b9552     1
49589dc19aec449f813a3486fc004af3     1
6af556ad6e3a4046b8eec8156295d9e9     1
Name: count, Length: 14173, dtype: int64

In [4]:
transcript["value/offer id"].value_counts()

value/offer id
fafdcd668e3743c1bb461111dcafc2a4    4555
f19421c1d4aa40978ebb69ca19b0e20d    4391
2298d6c36e964ae4a3e7e9706d1fb8c2    4229
4d5c57ea9a6940dd891ad53e9dbe8da0    4184
ae264e3637204a6fb9bb56bc8210ddfd    4154
5a8bc65990b245e5a138643cd4eb9837    4086
3f207df678b143eea3cee63160fa8bed    3454
2906b810c7d4411798c6938adc9daaa5    3395
9b98b8c7a33c4b65b9aebfe6a799e6d9    3338
0b1e1539f2cc45b7b9fa7c272da2e1d7    3095
Name: count, dtype: int64

**To merge the transcript and portfolio datasets, a common offer identifier is created
by aligning column names across both datasets.**


In [5]:
portfolio=portfolio.rename(columns={"id":"value/offer id"})

In [6]:
portfolio

,reward,channels/0,channels/1,channels/2,difficulty,duration,offer_type,value/offer id,channels/3
0,10,email,mobile,social,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,NaN
1,10,web,email,mobile,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,social
2,0,web,email,mobile,0,4,informational,3f207df678b143eea3cee63160fa8bed,NaN
3,5,web,email,mobile,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
4,5,web,email,NaN,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
5,3,web,email,mobile,7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,social
6,2,web,email,mobile,10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,social
7,0,email,mobile,social,0,3,informational,5a8bc65990b245e5a138643cd4eb9837,NaN
8,5,web,email,mobile,5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,social
9,2,web,email,mobile,10,7,discount,2906b810c7d4411798c6938adc9daaa5,NaN


**The transcript dataset represents event-level customer activity and is treated
as the primary dataset. Therefore, merging to enrich transcript
events with offer information from the portfolio dataset.**

In [7]:
#mergre between transcript and portfolio
df1=transcript.merge(portfolio,how="outer",on="value/offer id")

In [8]:
df1

,person,event,value/offer id,time,value/amount,value/offer_id,value/reward,reward,channels/0,channels/1,channels/2,difficulty,duration,offer_type,channels/3
0,a03223e636434f42ac4c3df47e8bac43,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,NaN,NaN,NaN,5.0,web,email,NaN,20.0,10.0,discount,NaN
1,aa4862eba776480b8bb9c68455b8c2e1,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,NaN,NaN,NaN,5.0,web,email,NaN,20.0,10.0,discount,NaN
2,31dda685af34476cad5bc968bdb01c53,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,NaN,NaN,NaN,5.0,web,email,NaN,20.0,10.0,discount,NaN
3,744d603ef08c4f33af5a61c8c7628d1c,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,NaN,NaN,NaN,5.0,web,email,NaN,20.0,10.0,discount,NaN
4,3d02345581554e81b7b289ab5e288078,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,NaN,NaN,NaN,5.0,web,email,NaN,20.0,10.0,discount,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59975,b311fa5c3dab428abc630f466e74302c,transaction,NaN,90,15.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59976,325a76dbd5de4f62b1230ea9659f26fa,transaction,NaN,90,33.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59977,33e4e9352ea04b559e0d0a210642ecb3,transaction,NaN,90,8.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59978,059de3bbf78b497fa190ecbc20d26521,transaction,NaN,90,10.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Next, customer demographic information is added by merging the profile dataset.
The merge is performed using a left join to ensure that all customer events
are preserved.**

**To merge df1 and profile we need to one column common  with two , so that i will rename name of id column in profile to person like in  df1 dataset**

In [9]:
profile=profile.rename(columns={"id":"person"})


In [10]:
profile

,gender,age,person,became_member_on,income
0,NaN,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,NaN,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,NaN,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN
...,...,...,...,...,...
16995,F,45,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0
16996,M,61,2cb4f97358b841b9a9773a7aa05a9d77,20180713,72000.0
16997,M,49,01d26f638c274aa0b965d24cefe3183f,20170126,73000.0
16998,F,83,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0


In [11]:
#mergre between all datasets
df_raw=profile.merge(df1,how="inner",on="person")

**After merging all datasets, the resulting dataframe contains event-level
customer activity enriched with both offer attributes and demographic
information.**

In [12]:
df_raw

,gender,age,person,became_member_on,income,event,value/offer id,time,value/amount,value/offer_id,value/reward,reward,channels/0,channels/1,channels/2,difficulty,duration,offer_type,channels/3
0,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0,transaction,NaN,18,21.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN
2,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,6,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN
4,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,6,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59975,F,62,e4052622e5ba45a8b96b59aba68cf068,20170722,82000.0,offer received,2298d6c36e964ae4a3e7e9706d1fb8c2,0,NaN,NaN,NaN,3.0,web,email,mobile,7.0,7.0,discount,social
59976,F,62,e4052622e5ba45a8b96b59aba68cf068,20170722,82000.0,offer viewed,2298d6c36e964ae4a3e7e9706d1fb8c2,6,NaN,NaN,NaN,3.0,web,email,mobile,7.0,7.0,discount,social
59977,F,62,e4052622e5ba45a8b96b59aba68cf068,20170722,82000.0,transaction,NaN,54,21.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59978,F,62,e4052622e5ba45a8b96b59aba68cf068,20170722,82000.0,offer completed,NaN,54,NaN,2298d6c36e964ae4a3e7e9706d1fb8c2,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**The number of rows after merging remains consistent with the transcript dataset,
ensuring that no event-level data is lost during the merge process.**

----------------------------

# Data Cleaning & Feature Engineering


A working copy of the merged dataset is created to preserve the original data and ensure safe, reproducible data cleaning.

In [13]:
# Create a working copy of the data
df = df_raw.copy()
print('Working copy created')

Working copy created


In [14]:
df.shape

(59980, 19)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59980 entries, 0 to 59979
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            52994 non-null  object 
 1   age               59980 non-null  int64  
 2   person            59980 non-null  object 
 3   became_member_on  59980 non-null  int64  
 4   income            52994 non-null  float64
 5   event             59980 non-null  object 
 6   value/offer id    38881 non-null  object 
 7   time              59980 non-null  int64  
 8   value/amount      16923 non-null  float64
 9   value/offer_id    4176 non-null   object 
 10  value/reward      4176 non-null   float64
 11  reward            38881 non-null  float64
 12  channels/0        38881 non-null  object 
 13  channels/1        38881 non-null  object 
 14  channels/2        35786 non-null  object 
 15  difficulty        38881 non-null  float64
 16  duration          38881 non-null  float6

___________________________________________________________________________________________________

The following data cleaning steps are performed:
1. Convert membership date to datetime format
2. Extract useful features from membership date
3. Drop unnecessary or redundant columns
4. Handle missing values
5. Remove duplicate records


> **1) Convert became_member_on to datetime**

**The `became_member_on` column is converted to datetime format to enable temporal feature extraction.
From this date, multiple time-based features are derived to better represent customer tenure.**


In [16]:
df.became_member_on

0        20170715
1        20170509
2        20170509
3        20170509
4        20170509
           ...   
59975    20170722
59976    20170722
59977    20170722
59978    20170722
59979    20170722
Name: became_member_on, Length: 59980, dtype: int64

In [17]:
#first convert to string to easy to insert - 
df['became_member'] = df['became_member_on'].astype(str)


#second insert - between year , month and day
def date(x):
    x=str(x)
    return x[:4]+"-"+x[4:6]+"-"+x[-2:]

#second insert - between year , month and day
df.became_member=df.became_member.apply(date)

df.became_member

0        2017-07-15
1        2017-05-09
2        2017-05-09
3        2017-05-09
4        2017-05-09
            ...    
59975    2017-07-22
59976    2017-07-22
59977    2017-07-22
59978    2017-07-22
59979    2017-07-22
Name: became_member, Length: 59980, dtype: object

In [18]:
#thrid convert to datetime
df.became_member=pd.to_datetime(df.became_member)


In [19]:
df.became_member

0       2017-07-15
1       2017-05-09
2       2017-05-09
3       2017-05-09
4       2017-05-09
           ...    
59975   2017-07-22
59976   2017-07-22
59977   2017-07-22
59978   2017-07-22
59979   2017-07-22
Name: became_member, Length: 59980, dtype: datetime64[ns]

In [20]:
df.columns

Index(['gender', 'age', 'person', 'became_member_on', 'income', 'event',
       'value/offer id', 'time', 'value/amount', 'value/offer_id',
       'value/reward', 'reward', 'channels/0', 'channels/1', 'channels/2',
       'difficulty', 'duration', 'offer_type', 'channels/3', 'became_member'],
      dtype='object')

In [21]:
#extract new features from became_member

df.insert(loc=4,value=df.became_member.dt.year,column="became_member_year") # year
df.insert(loc=5,value=df.became_member.dt.month,column="became_member_month") # month
df.insert(loc=6,value=df.became_member.dt.day,column="became_member_day") # day


In [22]:
df.head()

,gender,age,person,became_member_on,became_member_year,became_member_month,became_member_day,income,event,value/offer id,time,value/amount,value/offer_id,value/reward,reward,channels/0,channels/1,channels/2,difficulty,duration,offer_type,channels/3,became_member
0,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,2017,7,15,112000.0,transaction,NaN,18,21.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-15
1,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,2017,5,9,100000.0,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN,2017-05-09
2,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,2017,5,9,100000.0,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,6,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN,2017-05-09
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,2017,5,9,100000.0,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN,2017-05-09
4,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,2017,5,9,100000.0,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,6,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN,2017-05-09


In [23]:
min_day=df.became_member.min()
min_day

Timestamp('2013-07-29 00:00:00')

In [24]:
max_day=df.became_member.max()
max_day

Timestamp('2018-07-26 00:00:00')

In [25]:
df

,gender,age,person,became_member_on,became_member_year,became_member_month,became_member_day,income,event,value/offer id,time,value/amount,value/offer_id,value/reward,reward,channels/0,channels/1,channels/2,difficulty,duration,offer_type,channels/3,became_member
0,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,2017,7,15,112000.0,transaction,NaN,18,21.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-15
1,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,2017,5,9,100000.0,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN,2017-05-09
2,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,2017,5,9,100000.0,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,6,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN,2017-05-09
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,2017,5,9,100000.0,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN,2017-05-09
4,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,2017,5,9,100000.0,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,6,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN,2017-05-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59975,F,62,e4052622e5ba45a8b96b59aba68cf068,20170722,2017,7,22,82000.0,offer received,2298d6c36e964ae4a3e7e9706d1fb8c2,0,NaN,NaN,NaN,3.0,web,email,mobile,7.0,7.0,discount,social,2017-07-22
59976,F,62,e4052622e5ba45a8b96b59aba68cf068,20170722,2017,7,22,82000.0,offer viewed,2298d6c36e964ae4a3e7e9706d1fb8c2,6,NaN,NaN,NaN,3.0,web,email,mobile,7.0,7.0,discount,social,2017-07-22
59977,F,62,e4052622e5ba45a8b96b59aba68cf068,20170722,2017,7,22,82000.0,transaction,NaN,54,21.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-22
59978,F,62,e4052622e5ba45a8b96b59aba68cf068,20170722,2017,7,22,82000.0,offer completed,NaN,54,NaN,2298d6c36e964ae4a3e7e9706d1fb8c2,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-22


**A new feature `login_days` is created to represent how long a customer has been a member of the Starbucks rewards program.
This feature captures customer loyalty and engagement duration.**


In [26]:
# extract new feature to show how long customer download the application 

max_day=df.became_member.max()

value=(max_day-df.became_member).dt.days
df.insert(loc=7,value=value,column="login_days") # login_days



In [27]:
# now we didnt need became_member_on 
df.drop("became_member_on",axis=1,inplace=True)

In [28]:
df.head()

,gender,age,person,became_member_year,became_member_month,became_member_day,login_days,income,event,value/offer id,time,value/amount,value/offer_id,value/reward,reward,channels/0,channels/1,channels/2,difficulty,duration,offer_type,channels/3,became_member
0,F,55,0610b486422d4921ae7d2bf64640c50b,2017,7,15,376,112000.0,transaction,NaN,18,21.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-15
1,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017,5,9,443,100000.0,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN,2017-05-09
2,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017,5,9,443,100000.0,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,6,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN,2017-05-09
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017,5,9,443,100000.0,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN,2017-05-09
4,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017,5,9,443,100000.0,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,6,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN,2017-05-09


**After extracting all relevant time-based features, the original `became_member_on` column is dropped to avoid redundancy.**

-------------------------

> **2) Drop unnecessary columns**

**Some columns are dropped because they are either redundant, unused, or
represent intermediate values that are no longer required.**

In [29]:
df

,gender,age,person,became_member_year,became_member_month,became_member_day,login_days,income,event,value/offer id,time,value/amount,value/offer_id,value/reward,reward,channels/0,channels/1,channels/2,difficulty,duration,offer_type,channels/3,became_member
0,F,55,0610b486422d4921ae7d2bf64640c50b,2017,7,15,376,112000.0,transaction,NaN,18,21.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-15
1,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017,5,9,443,100000.0,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN,2017-05-09
2,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017,5,9,443,100000.0,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,6,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN,2017-05-09
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017,5,9,443,100000.0,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN,2017-05-09
4,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017,5,9,443,100000.0,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,6,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN,2017-05-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59975,F,62,e4052622e5ba45a8b96b59aba68cf068,2017,7,22,369,82000.0,offer received,2298d6c36e964ae4a3e7e9706d1fb8c2,0,NaN,NaN,NaN,3.0,web,email,mobile,7.0,7.0,discount,social,2017-07-22
59976,F,62,e4052622e5ba45a8b96b59aba68cf068,2017,7,22,369,82000.0,offer viewed,2298d6c36e964ae4a3e7e9706d1fb8c2,6,NaN,NaN,NaN,3.0,web,email,mobile,7.0,7.0,discount,social,2017-07-22
59977,F,62,e4052622e5ba45a8b96b59aba68cf068,2017,7,22,369,82000.0,transaction,NaN,54,21.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-22
59978,F,62,e4052622e5ba45a8b96b59aba68cf068,2017,7,22,369,82000.0,offer completed,NaN,54,NaN,2298d6c36e964ae4a3e7e9706d1fb8c2,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-22


In [30]:
df.columns

Index(['gender', 'age', 'person', 'became_member_year', 'became_member_month',
       'became_member_day', 'login_days', 'income', 'event', 'value/offer id',
       'time', 'value/amount', 'value/offer_id', 'value/reward', 'reward',
       'channels/0', 'channels/1', 'channels/2', 'difficulty', 'duration',
       'offer_type', 'channels/3', 'became_member'],
      dtype='object')

In [31]:
df.drop(['value/amount', 'value/offer_id', 'value/reward','channels/0', 'channels/1', 'channels/2','channels/3'],axis=1,inplace=True)

In [32]:
df

,gender,age,person,became_member_year,became_member_month,became_member_day,login_days,income,event,value/offer id,time,reward,difficulty,duration,offer_type,became_member
0,F,55,0610b486422d4921ae7d2bf64640c50b,2017,7,15,376,112000.0,transaction,NaN,18,NaN,NaN,NaN,NaN,2017-07-15
1,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017,5,9,443,100000.0,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,5.0,5.0,7.0,bogo,2017-05-09
2,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017,5,9,443,100000.0,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,6,5.0,5.0,7.0,bogo,2017-05-09
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017,5,9,443,100000.0,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,5.0,5.0,7.0,bogo,2017-05-09
4,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017,5,9,443,100000.0,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,6,5.0,5.0,7.0,bogo,2017-05-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59975,F,62,e4052622e5ba45a8b96b59aba68cf068,2017,7,22,369,82000.0,offer received,2298d6c36e964ae4a3e7e9706d1fb8c2,0,3.0,7.0,7.0,discount,2017-07-22
59976,F,62,e4052622e5ba45a8b96b59aba68cf068,2017,7,22,369,82000.0,offer viewed,2298d6c36e964ae4a3e7e9706d1fb8c2,6,3.0,7.0,7.0,discount,2017-07-22
59977,F,62,e4052622e5ba45a8b96b59aba68cf068,2017,7,22,369,82000.0,transaction,NaN,54,NaN,NaN,NaN,NaN,2017-07-22
59978,F,62,e4052622e5ba45a8b96b59aba68cf068,2017,7,22,369,82000.0,offer completed,NaN,54,NaN,NaN,NaN,NaN,2017-07-22


--------------------------

> **3) Handle missing values**

**Missing values are handled carefully to avoid losing customer records while
preserving the integrity of the dataset.**

In [33]:
round(df.isna().sum()/df.shape[0],2)*100

gender                 12.0
age                     0.0
person                  0.0
became_member_year      0.0
became_member_month     0.0
became_member_day       0.0
login_days              0.0
income                 12.0
event                   0.0
value/offer id         35.0
time                    0.0
reward                 35.0
difficulty             35.0
duration               35.0
offer_type             35.0
became_member           0.0
dtype: float64

In [34]:
df.gender.value_counts()

gender
M    29847
F    22315
O      832
Name: count, dtype: int64

In [35]:
df[df.gender.isna()]

,gender,age,person,became_member_year,became_member_month,became_member_day,login_days,income,event,value/offer id,time,reward,difficulty,duration,offer_type,became_member
5,NaN,118,a03223e636434f42ac4c3df47e8bac43,2017,8,4,356,NaN,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,5.0,20.0,10.0,discount,2017-08-04
6,NaN,118,a03223e636434f42ac4c3df47e8bac43,2017,8,4,356,NaN,offer viewed,0b1e1539f2cc45b7b9fa7c272da2e1d7,6,5.0,20.0,10.0,discount,2017-08-04
7,NaN,118,a03223e636434f42ac4c3df47e8bac43,2017,8,4,356,NaN,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,5.0,20.0,10.0,discount,2017-08-04
8,NaN,118,a03223e636434f42ac4c3df47e8bac43,2017,8,4,356,NaN,offer viewed,0b1e1539f2cc45b7b9fa7c272da2e1d7,6,5.0,20.0,10.0,discount,2017-08-04
12,NaN,118,8ec6ce2a7e7949b1bf142def7d0e0586,2017,9,25,304,NaN,offer received,fafdcd668e3743c1bb461111dcafc2a4,0,2.0,10.0,10.0,discount,2017-09-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59960,NaN,118,c02b10e8752c4d8e9b73f918558531f7,2015,12,11,958,NaN,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,5.0,20.0,10.0,discount,2015-12-11
59961,NaN,118,c02b10e8752c4d8e9b73f918558531f7,2015,12,11,958,NaN,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,5.0,20.0,10.0,discount,2015-12-11
59962,NaN,118,c02b10e8752c4d8e9b73f918558531f7,2015,12,11,958,NaN,offer viewed,0b1e1539f2cc45b7b9fa7c272da2e1d7,54,5.0,20.0,10.0,discount,2015-12-11
59963,NaN,118,c02b10e8752c4d8e9b73f918558531f7,2015,12,11,958,NaN,transaction,NaN,0,NaN,NaN,NaN,NaN,2015-12-11


- note that all missing value in gender same missing values in income ,sothat i will drop thoes values

In [36]:
df.dropna(subset=["gender"],inplace=True)

In [37]:
#check
round(df.isna().sum()/df.shape[0],2)*100

gender                  0.0
age                     0.0
person                  0.0
became_member_year      0.0
became_member_month     0.0
became_member_day       0.0
login_days              0.0
income                  0.0
event                   0.0
value/offer id         36.0
time                    0.0
reward                 36.0
difficulty             36.0
duration               36.0
offer_type             36.0
became_member           0.0
dtype: float64

In [38]:
df[df["value/offer id"].isna()]

,gender,age,person,became_member_year,became_member_month,became_member_day,login_days,income,event,value/offer id,time,reward,difficulty,duration,offer_type,became_member
0,F,55,0610b486422d4921ae7d2bf64640c50b,2017,7,15,376,112000.0,transaction,NaN,18,NaN,NaN,NaN,NaN,2017-07-15
23,M,65,389bc3fa690240e798340f5a15918d5c,2018,2,9,167,53000.0,transaction,NaN,60,NaN,NaN,NaN,NaN,2018-02-09
24,M,65,389bc3fa690240e798340f5a15918d5c,2018,2,9,167,53000.0,offer completed,NaN,60,NaN,NaN,NaN,NaN,2018-02-09
38,M,26,e12aeaf2d47d42479ea1c4ac3d8286c6,2014,2,13,1624,46000.0,transaction,NaN,36,NaN,NaN,NaN,NaN,2014-02-13
39,M,26,e12aeaf2d47d42479ea1c4ac3d8286c6,2014,2,13,1624,46000.0,transaction,NaN,90,NaN,NaN,NaN,NaN,2014-02-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59966,F,45,6d5f3a774f3d4714ab0c092238f3a1d7,2018,6,4,52,54000.0,transaction,NaN,84,NaN,NaN,NaN,NaN,2018-06-04
59972,F,83,9dc1421481194dcd9400aec7c9ae6366,2016,3,7,871,50000.0,transaction,NaN,24,NaN,NaN,NaN,NaN,2016-03-07
59977,F,62,e4052622e5ba45a8b96b59aba68cf068,2017,7,22,369,82000.0,transaction,NaN,54,NaN,NaN,NaN,NaN,2017-07-22
59978,F,62,e4052622e5ba45a8b96b59aba68cf068,2017,7,22,369,82000.0,offer completed,NaN,54,NaN,NaN,NaN,NaN,2017-07-22


missing values in [value/offer id ,reward ,difficulty ,duration and offer_type] is represended to a customer doesn't have offer

i will put all missing values by consent value equal 0

In [39]:
df.fillna(0,inplace=True)

In [40]:
#check
round(df.isna().sum()/df.shape[0],2)*100

gender                 0.0
age                    0.0
person                 0.0
became_member_year     0.0
became_member_month    0.0
became_member_day      0.0
login_days             0.0
income                 0.0
event                  0.0
value/offer id         0.0
time                   0.0
reward                 0.0
difficulty             0.0
duration               0.0
offer_type             0.0
became_member          0.0
dtype: float64

In [41]:
df['offer_type'] = df['offer_type'].replace({0: 'no_offer'})

**Missing offer-related values indicate that a customer did not receive an offer
during a specific event. These values are preserved and handled during
aggregation.**

-----------------------

> **4) Check & Remove duplicates records**

In [42]:
df.duplicated().sum()
df[df.duplicated()]


,gender,age,person,became_member_year,became_member_month,became_member_day,login_days,income,event,value/offer id,time,reward,difficulty,duration,offer_type,became_member
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017,5,9,443,100000.0,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,5.0,5.0,7.0,bogo,2017-05-09
4,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017,5,9,443,100000.0,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,6,5.0,5.0,7.0,bogo,2017-05-09
10,M,68,e2127556f4f64592b11af22de27a7932,2018,4,26,91,70000.0,offer received,2906b810c7d4411798c6938adc9daaa5,0,2.0,10.0,7.0,discount,2018-04-26
21,M,65,389bc3fa690240e798340f5a15918d5c,2018,2,9,167,53000.0,offer received,f19421c1d4aa40978ebb69ca19b0e20d,0,5.0,5.0,5.0,bogo,2018-02-09
22,M,65,389bc3fa690240e798340f5a15918d5c,2018,2,9,167,53000.0,offer viewed,f19421c1d4aa40978ebb69ca19b0e20d,0,5.0,5.0,5.0,bogo,2018-02-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59959,M,60,cb23b66c56f64b109d673d5e56574529,2018,5,5,82,113000.0,offer received,2906b810c7d4411798c6938adc9daaa5,0,2.0,10.0,7.0,discount,2018-05-05
59968,M,61,2cb4f97358b841b9a9773a7aa05a9d77,2018,7,13,13,72000.0,offer received,5a8bc65990b245e5a138643cd4eb9837,0,0.0,0.0,3.0,informational,2018-07-13
59971,M,49,01d26f638c274aa0b965d24cefe3183f,2017,1,26,546,73000.0,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,5.0,20.0,10.0,discount,2017-01-26
59975,F,62,e4052622e5ba45a8b96b59aba68cf068,2017,7,22,369,82000.0,offer received,2298d6c36e964ae4a3e7e9706d1fb8c2,0,3.0,7.0,7.0,discount,2017-07-22


In [43]:
before = df.shape[0]
df = df.drop_duplicates()
after = df.shape[0]
print(f'Removed {before - after} duplicate rows')

Removed 17498 duplicate rows


In [44]:
before


52994

In [45]:
after

35496

**Duplicate rows are removed to ensure data consistency.**

--------------------

<a id=section4></a>

In [46]:
df.isnull().sum(), df.shape

(gender                 0
 age                    0
 person                 0
 became_member_year     0
 became_member_month    0
 became_member_day      0
 login_days             0
 income                 0
 event                  0
 value/offer id         0
 time                   0
 reward                 0
 difficulty             0
 duration               0
 offer_type             0
 became_member          0
 dtype: int64,
 (35496, 16))

# Save Final Dataset

**The final cleaned and aggregated dataset is saved for downstream modeling.**

In [47]:
df.to_csv("../../03_Data/Processed_Data/final_Cleaned_Data.csv",index=False)
print('Clean & safe dataset saved successfully!')

Clean & safe dataset saved successfully!
